In [ ]:
#Dependencies and api_keys
import requests
from pprint import pprint
from config import census_key
import json

In [22]:
# Create list of state codes
state_codes = []
for number in range(1, 10):
    state_codes.append('0{0}'.format(number))
for number in range(11,52):
    state_codes.append('{0}'.format(number))
state_codes[0]

'01'

In [39]:
# Loop through state code list, skipping states without data
census_master = []
fail_master = []
for code in state_codes:
    try:
        census_test = requests.get('https://api.census.gov/data/2009/acs/flows?get=MOVEDIN,MOVEDOUT,GEOID2,GEOID1,MOVEDNET,FULL2_NAME,FULL1_NAME' +
                               '&for=county:*&in=state:{0}&key='.format(code) +
                               census_key).json()
        census_master.append(census_test)
    except:
        fail_master.append(code)

In [40]:
fail_master

['03', '07', '14', '43']